In [6]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

import sys
sys.path.append('../')
from config import Config, Shape
from plate import get_base, get_screw_positions, get_key_positions, make_plate
from case import get_basic_shape,  cut_aviator_connector_hole, get_distance_between_two_vertices,  add_microcontrollerbox


Enabling jupyter_cadquery replay


In [7]:
def make_controller_box_top_plate(case,config):

    controllerBoxLength =  get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<X'))['y']
    controllerBoxWidth = get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<Y'))['x']

    controller_box_top = (case
                        .faces(tag='controllerBoxTop')
                        .workplane(centerOption="CenterOfBoundBox", invert=True)
                        .move(0,config.wallThickness)
                        .box(controllerBoxWidth,controllerBoxLength, config.controllerBoxThickness, centered=[True,True,False],combine=False)
    )


    return controller_box_top

In [2]:

rkn = [4,8,8,6]
config=Config(rkn, shape=Shape.LEAN, floorThickness=3)
kp = get_key_positions(config)

In [79]:
case = make_case(config)

if True:
    controllerBoxLength =  get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<X'))['y']
    controllerBoxWidth = get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<Y'))['x']+10

    controller_box_top = (case
                        .faces(tag='controllerBoxTop')
                        .workplane(centerOption="CenterOfBoundBox", invert=True)
                        .move(0,config.wallThickness)
                        .box(controllerBoxWidth,controllerBoxLength, config.controllerBoxThickness, centered=[True,True,False],combine=False)
    )

#case.intersect(controller_box_top)
    


In [52]:
case.edges(">Z and (not #X) and (not #Y)")#.vertices()


100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [68]:
case= get_basic_shape(config)
case.faces('<Z or >Z').tag('bottomface')
case.faces('+X or +Y or -X or -Y').tag('outface')

case

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [73]:
case.faces(tag= 'outface')


100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.08s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [82]:
case.faces(tag='outface').faces(cq.selectors.TypeSelector("CYLINDER"))

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.03s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [86]:
case.faces(tag='outface').faces('+Y and >>Y[-2]')

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [78]:


def make_case(config:Config) -> cq.Sketch:
    """TODO: add screw holes for microcontroller (check they are in right position!!)
            change fillet parameters
            add module for leds, encoder and aviator
            """
    
    
    case= get_basic_shape(config)
    case.faces('<Z or >Z').tag('bottomface')


    # add box for microcontroller
    if config.controller is not None:
        case = add_microcontrollerbox(case, config)


    #fillet edges
    case = case.edges('|Z').fillet(config.edgeFillet)
    #case.edges('>Z').tag('outerTopEdge')
    case.faces('(not <Z) and (not >Z)').tag('outface')


    # scoop out interior
    case= (case.faces(">Z or <Z").shell(-config.wallThickness, kind='intersection'))
    
    # set floor thickness 
    case = case.faces('<Z').wires('>X and >Y').toPending().extrude(config.floorThickness)
    
    return case
    
    #fillet bottom edge
    case = case.edges('<Z').fillet(3)



    # add plate screw holes 
    scr_hls = get_screw_positions(config)
    case=(case.faces('<Z').workplane(origin=(0,0))
          .pushPoints([(x,-y) for x,y in scr_hls])
          .cskHole(2.4, 4.8, 82, depth=None)    
          )

    # add controllerbox holes.
    # TODO - remove hardcoding on screw hole size
    if config.controller is not None:
        case = (case.workplaneFromTagged("controllerBox")
                .rect(config.controller.screw_hole_x,
                        config.controller.screw_hole_y- config.wallThickness - config.controllerYOffset - (config.controller.board_dimension_y -config.controller.screw_hole_y)*0.5, 
                        forConstruction=True ,centered=[True,True,True])
                .vertices()
                .cskHole(2.4, 4.8, 82, depth=None))
    
    #cut hole for aviator connector 
    case = cut_aviator_connector_hole(case, config)

    return case


In [11]:
case= get_basic_shape(config)
    #case.faces('<Z').tag('bottomface')

    # add box for microcontroller
if config.controller is not None:
        case = add_microcontrollerbox(case, config)
    
case= (case.faces(">Z or <Z").shell(-config.wallThickness, kind='intersection'))



In [25]:
casse = case.edges('|Z').fillet(2)

{'x': 56.0, 'y': 0.0, 'z': 0.0}